In [1]:
import torch
import torch.nn.functional
dtype = torch.cuda.FloatTensor
dtype_long = torch.cuda.LongTensor
import time
import numpy as np
torch.manual_seed(0)

In [2]:
dim= 5 # Size of Image

In [3]:
image = torch.rand(1, 2, dim, dim) #BCYX
print("image=", image)

image= tensor([[[[0.4963, 0.7682, 0.0885, 0.1320, 0.3074],
          [0.6341, 0.4901, 0.8964, 0.4556, 0.6323],
          [0.3489, 0.4017, 0.0223, 0.1689, 0.2939],
          [0.5185, 0.6977, 0.8000, 0.1610, 0.2823],
          [0.6816, 0.9152, 0.3971, 0.8742, 0.4194]],

         [[0.5529, 0.9527, 0.0362, 0.1852, 0.3734],
          [0.3051, 0.9320, 0.1759, 0.2698, 0.1507],
          [0.0317, 0.2081, 0.9298, 0.7231, 0.7423],
          [0.5263, 0.2437, 0.5846, 0.0332, 0.1387],
          [0.2422, 0.8155, 0.7932, 0.2783, 0.4820]]]])


In [4]:
xm = torch.linspace(0, 1, dim).view(
            1, 1, -1).expand(1, dim, dim)
ym = torch.linspace(0, 1, dim).view(
            1, -1, 1).expand(1, dim, dim)
xym = torch.cat((ym, xm), 0)
xym.unsqueeze_(0)
print("xym shape is ", xym.shape)
print("xym", xym)

xym shape is  torch.Size([1, 2, 5, 5])
xym tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.2500, 0.2500, 0.2500, 0.2500, 0.2500],
          [0.5000, 0.5000, 0.5000, 0.5000, 0.5000],
          [0.7500, 0.7500, 0.7500, 0.7500, 0.7500],
          [1.0000, 1.0000, 1.0000, 1.0000, 1.0000]],

         [[0.0000, 0.2500, 0.5000, 0.7500, 1.0000],
          [0.0000, 0.2500, 0.5000, 0.7500, 1.0000],
          [0.0000, 0.2500, 0.5000, 0.7500, 1.0000],
          [0.0000, 0.2500, 0.5000, 0.7500, 1.0000],
          [0.0000, 0.2500, 0.5000, 0.7500, 1.0000]]]])


In [5]:
result = xym + torch.tanh(image)
print("result\n", result)

result
 tensor([[[[0.4592, 0.6459, 0.0882, 0.1313, 0.2981],
          [0.8109, 0.7043, 0.9646, 0.6765, 0.8096],
          [0.8354, 0.8814, 0.5223, 0.6673, 0.7857],
          [1.2266, 1.3529, 1.4140, 0.9097, 1.0250],
          [1.5926, 1.7236, 1.3775, 1.7035, 1.3964]],

         [[0.5027, 0.9910, 0.5361, 0.9331, 1.3570],
          [0.2960, 0.9815, 0.6741, 1.0135, 1.1495],
          [0.0317, 0.4552, 1.2305, 1.3688, 1.6306],
          [0.4825, 0.4889, 1.0260, 0.7831, 1.1378],
          [0.2376, 0.9226, 1.1602, 1.0213, 1.4478]]]])


### PYTORCH

In [6]:
result_corrected=result*2-1 # result_corrected is between -1 and 1
predictions=torch.nn.functional.grid_sample(image, result_corrected.permute(0, 2, 3,1), align_corners=True)
print("interpolated predictions \n", predictions)

interpolated predictions 
 tensor([[[[0.0918, 0.6666, 0.3985, 0.7530, 0.0000],
          [0.4436, 0.7493, 0.2684, 0.6944, 0.3076],
          [0.2330, 0.2909, 0.0343, 0.0000, 0.0000],
          [0.0298, 0.0000, 0.0000, 0.2842, 0.1694],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.8089, 0.4844, 0.1421, 0.4990, 0.0000],
          [0.3304, 0.3548, 0.3107, 0.4065, 0.1313],
          [0.2469, 0.6389, 0.0583, 0.0000, 0.0000],
          [0.0657, 0.0000, 0.0000, 0.1414, 0.1946],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]]])



### SCIPY

In [7]:
from scipy import interpolate
f = interpolate.interp2d(np.linspace(0, 1, dim), np.linspace(0, 1, dim), image[0, 0, ...].cpu().detach().numpy(), kind='linear', fill_value=0)
g = interpolate.interp2d(np.linspace(0, 1, dim), np.linspace(0, 1, dim), image[0, 1, ...].cpu().detach().numpy(), kind='linear', fill_value=0)
scipy_predictions= np.zeros((2, dim, dim))
for i in range(result.shape[2]):
    for j in range(result.shape[3]):
        scipy_predictions[0, i, j]= f(result[0, 0, i, j], result[0, 1, i, j])
        scipy_predictions[1, i, j]= g(result[0, 0, i, j], result[0, 1, i, j])

In [8]:
print("interpolated scipy predictions \n", scipy_predictions)

interpolated scipy predictions 
 [[[0.09182841 0.66657711 0.39851493 0.75299857 0.        ]
  [0.44358824 0.74931196 0.26844675 0.         0.        ]
  [0.23303955 0.29086799 0.         0.         0.        ]
  [0.         0.         0.         0.28422691 0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.80887856 0.48441457 0.1420741  0.499009   0.        ]
  [0.33037722 0.35477801 0.31068349 0.         0.        ]
  [0.24692629 0.63890268 0.         0.         0.        ]
  [0.         0.         0.         0.14136588 0.        ]
  [0.         0.         0.         0.         0.        ]]]


### NUMPY 

In [10]:
numpy_predictions=np.zeros((2, dim, dim))
for y in range(result.shape[2]):
    for x in range(result.shape[3]):

        y_l=result[0, 0, y, x] // (1/(dim-1)) # 0, 1
        x_l=result[0, 1, y, x] // (1/(dim-1)) # 0, 1
        
        y_u=y_l+1
        x_u=x_l+1
        
        y0 = int(np.clip(y_l, 0, result.shape[2]-1))
        y1 = int(np.clip(y_u, 0, result.shape[2]-1))
        x0 = int(np.clip(x_l, 0, result.shape[3]-1))
        x1 = int(np.clip(x_u, 0, result.shape[3]-1))
        
        
        Ia = image[0, 0, y0, x0 ] # tl
        Ib = image[0, 0, y1, x0 ] # bl
        Ic = image[0, 0, y0, x1 ] # tr
        Id = image[0, 0, y1, x1 ] # br

        ys=result[0, 0, y, x] 
        xs=result[0, 1, y, x]
        

        a= np.array([y1*1/(dim-1)-ys, ys - 1/(dim-1)*y0])
        b= np.array([[Ia, Ib], [Ic, Id]])
        c= np.array([[x1*1/(dim-1)-xs],[xs-x0*1/(dim-1)]])

        numpy_predictions[0, y, x]=1/(1/(dim-1)*1/(dim-1))*np.matmul(a, np.matmul(b, c))


        Ia = image[0, 1, y0, x0 ]
        Ib = image[0, 1, y1, x0 ]
        Ic = image[0, 1, y0, x1 ]
        Id = image[0, 1, y1, x1 ]

        a= np.array([y1*1/(dim-1)-ys, ys - 1/(dim-1)*y0])
        b= np.array([[Ia, Ib], [Ic, Id]])
        c= np.array([[x1*1/(dim-1)-xs],[xs-x0*1/(dim-1)]])

        numpy_predictions[1, y, x]=1/(1/(dim-1)*1/(dim-1))*np.matmul(a, np.matmul(b, c))
        
        
print(numpy_predictions.astype(np.float16))

[[[ 5.2344e-01  2.3218e-01  1.9592e-01  4.6167e-01 -4.8315e-01]
  [ 7.3486e-01  2.6099e-01  6.3818e-01 -6.2609e-04  8.2031e-02]
  [ 6.0254e-01  6.6211e-01 -1.0712e-02 -1.7136e-02  3.4595e-01]
  [-4.6948e-01 -7.3145e-01  0.0000e+00  2.8418e-01  0.0000e+00]
  [ 5.5371e-01 -1.3164e+00  0.0000e+00  0.0000e+00  0.0000e+00]]

 [[ 2.6001e-01  1.1774e-01  1.0620e-01  2.2778e-01  8.4473e-01]
  [ 4.1553e-01  1.6516e-01  2.7832e-01 -3.2532e-02  2.0532e-01]
  [ 4.3091e-01  7.3535e-01 -5.5664e-01 -8.9062e-01  8.6572e-01]
  [-2.0218e-02 -3.1494e-02  0.0000e+00  1.4136e-01  0.0000e+00]
  [ 1.3584e+00  5.8984e-01  0.0000e+00  0.0000e+00  0.0000e+00]]]


In [11]:
print(scipy_predictions)

[[[0.09182841 0.66657711 0.39851493 0.75299857 0.        ]
  [0.44358824 0.74931196 0.26844675 0.         0.        ]
  [0.23303955 0.29086799 0.         0.         0.        ]
  [0.         0.         0.         0.28422691 0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[0.80887856 0.48441457 0.1420741  0.499009   0.        ]
  [0.33037722 0.35477801 0.31068349 0.         0.        ]
  [0.24692629 0.63890268 0.         0.         0.        ]
  [0.         0.         0.         0.14136588 0.        ]
  [0.         0.         0.         0.         0.        ]]]
